# Validation

In [36]:
! pip install einops
! pip install imutils
! pip install opencv-python
! pip install matplotlib
! pip install scikit-image
! pip install natsort
! pip install numpy
! pip install kornia
! pip install torch
! pip install kornia moons
! pip install torchvision

ERROR: Could not find a version that satisfies the requirement moons (from versions: none)
ERROR: No matching distribution found for moons


In [37]:
import os
from collections import namedtuple
import cv2
import csv
import kornia as K
import kornia.feature as KF
from kornia.feature.loftr import LoFTR
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import glob
import random
import imutils
from tqdm.notebook import tqdm
from PIL import Image
import matplotlib.cm as cm

from kornia_moons.feature import *

# from src.loftr import LoFTR, default_cfg
from src.utils.plotting import make_matching_figure

# =============================

import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte

from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np
from numpy.lib.arraypad import pad

# =================================================================
import os
import numpy as np
import cv2
import csv
from glob import glob
import torch
import matplotlib.pyplot as plt
import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF
import gc
import shutil

In [38]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.to(device).eval()

In [39]:
LONGEST_EDGE = 1280
def resize_keep_ratio(img, longest_size=LONGEST_EDGE):
    height, width = img.shape[:2]
    if np.maximum(height, width) <= longest_size: # no need to resize
        return img
    
    if height >= width:
        resized_img = imutils.resize(img, height=longest_size)
    else:
        resized_img = imutils.resize(img, width=longest_size)
    return resized_img

def load_torch_image(fname, device):
    img = cv2.imread(fname)
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img.to(device)

In [40]:
def match(img_path0, img_path1, matcher, device=device):
    img0 = load_torch_image(img_path0)
    img1 = load_torch_image(img_path1)
        
    input_dict = {"image0": K.color.rgb_to_grayscale(img0).to(device), 
                  "image1": K.color.rgb_to_grayscale(img1).to(device)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
        
    return mkpts0, mkpts1
        
def get_F_matrix(mkpts0, mkpts1):

    # Make sure we do not trigger an exception here.
    if len(mkpts0) > 8:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)

        assert F.shape == (3, 3), 'Malformed F?'
    else:
        F = np.zeros((3, 3))

    return F

In [41]:
src = '../image-matching-challenge-2022/train'

scaling_dict = {}
with open(f'{src}/scaling_factors.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        scaling_dict[row[0]] = float(row[1])

print(f'Scaling factors: {scaling_dict}')

Scaling factors: {'british_museum': 2.517, 'brandenburg_gate': 7.38, 'buckingham_palace': 18.75, 'colosseum_exterior': 36.99, 'grand_place_brussels': 10.26, 'lincoln_memorial_statue': 1.85, 'notre_dame_front_facade': 1.36, 'pantheon_exterior': 5.41, 'piazza_san_marco': 7.92, 'sacre_coeur': 20.27, 'sagrada_familia': 4.2, 'st_pauls_cathedral': 7.01, 'st_peters_square': 21.48, 'taj_mahal': 20.76, 'temple_nara_japan': 7.79, 'trevi_fountain': 3.67}


In [47]:
def restoration(img_pth0, img_pth1):
    # task = 'Real_Denoising'
    task = 'Single_Image_Defocus_Deblurring'
    # task = 'Motion_Deblurring'
    # task = 'Deraining'

    shutil.rmtree('demo')

    input_dir = 'demo/sample_images/'+task+'/degraded'
    os.makedirs(input_dir, exist_ok=True)
    
    shutil.copy(img_pth0, input_dir)
    shutil.copy(img_pth1, input_dir)

    def get_weights_and_parameters(task, parameters):
        if task == 'Motion_Deblurring':
            weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
        elif task == 'Single_Image_Defocus_Deblurring':
            weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
        elif task == 'Deraining':
            weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
        elif task == 'Real_Denoising':
            weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
            parameters['LayerNorm_type'] =  'BiasFree'
        return weights, parameters

    # Get model weights and parameters
    parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
    weights, parameters = get_weights_and_parameters(task, parameters)

    load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
    model = load_arch['Restormer'](**parameters)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(device)
    # print(device)
    checkpoint = torch.load(weights)
    # print(checkpoint)
    model.load_state_dict(checkpoint['params'])
    model = model.to(device).eval()

    input_dir = 'demo/sample_images/'+task+'/degraded'
    out_dir = 'demo/sample_images/'+task+'/restored'
    os.makedirs(out_dir, exist_ok=True)
    extensions = ['jpg', 'JPG', 'png', 'PNG', 'jpeg', 'JPEG', 'bmp', 'BMP']
    files = natsorted(glob(os.path.join(input_dir, '*')))

    img_multiple_of = 8

    print(f"\n ==> Running {task} with weights {weights}\n ")
    with torch.no_grad():
        for filepath in tqdm(files):
            img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
            input_ = torch.from_numpy(img).float().div(255.).permute(2, 0, 1).unsqueeze(0)

            # Pad the input if not_multiple_of 8
            h, w = input_.shape[2], input_.shape[3]
            H, W = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of, ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0

            # Convert input_ to NumPy array
            input_ = input_.squeeze(0).permute(1, 2, 0).cpu().numpy()
            input_ = np.pad(input_, ((0, padh), (0, padw), (0, 0)), mode='reflect')
            input_ = torch.from_numpy(input_).permute(2, 0, 1).unsqueeze(0)

            restored = model(input_)
            restored = torch.clamp(restored, 0, 1)

            # Unpad the output
            restored = restored[:, :, :h, :w]

            restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
            restored = img_as_ubyte(restored[0])

            filename = os.path.split(filepath)[-1]
            cv2.imwrite(os.path.join(out_dir, filename), cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))    
    
    new_img_pth0 = out_dir + '/' + os.listdir(out_dir)[0]
    new_img_pth1 = out_dir + '/' + os.listdir(out_dir)[1]
    print(os.listdir(out_dir)[0] + ' RESTORED')
    print(os.listdir(out_dir)[1] + ' RESTORED')
    return new_img_pth0, new_img_pth1

In [48]:
from imc_metric import EvaluateSubmission, ReadCovisibilityData, FlattenMatrix

In [49]:
# max_num_pairs = 2

# sample_ids = []
# fund_matrices = []

# count = 0
# scalingLen = len(scaling_dict.keys())

# for scene in scaling_dict.keys():
#     # print(scene)
#     count += 1
#     print('Folder ' + str(count) + '/' + str(scalingLen))

#     covisibility_dict, F_gt_dict = ReadCovisibilityData(f'{src}/{scene}/pair_covisibility.csv')

#     # Let's remove pairs with a covisibility below 0.1. Note that the keys are roughly sorted by difficulty, in increasing order, so let's also shuffle before subsampling.
#     # Neither matters for the purposes of this exercise, but let's prevent mistakes down the line.
#     pairs = list([key for key, covis in covisibility_dict.items() if covis >= 0.7])
#     random.shuffle(pairs)
#     n = len(pairs)
#     pairs = pairs[:max_num_pairs]
#     # print(pairs)
#     print(f'Loading covisibility data for "{scene}"... kept {len(pairs)} out of {n} covisible pairs')
    
#     for pair in tqdm(pairs):
#         image_0_id, image_1_id = pair.split('-')
#         img_path0 = f'{src}/{scene}/images/{image_0_id}.jpg'
#         img_path1 = f'{src}/{scene}/images/{image_1_id}.jpg'

#         # img_path0, img_path1 = restoration(img_path0, img_path1)

#         mkpts0, mkpts1 = match(img_path0, img_path1, matcher)
#         F = get_F_matrix(mkpts0, mkpts1)
#         F_str = FlattenMatrix(F)

#         sample_ids.append(f'phototourism;{scene};{pair}')
#         fund_matrices.append(F_str)
#         # break
#     if count == 2:
#         break
#     # break

In [52]:
max_num_pairs = 2

F_dict = {}

sample_ids = []
fund_matrices = []

count = 0
folder_limit = 16
scalingLen = len(scaling_dict.keys())

import time
for scene in scaling_dict.keys():
    count += 1
    print('Folder ' + str(count) + '/' + str(scalingLen))

    covisibility_dict, F_gt_dict = ReadCovisibilityData(f'{src}/{scene}/pair_covisibility.csv')

    # Let's remove pairs with a covisibility below 0.1. Note that the keys are roughly sorted by difficulty, in increasing order, so let's also shuffle before subsampling.
    # Neither matters for the purposes of this exercise, but let's prevent mistakes down the line.
    pairs = list([key for key, covis in covisibility_dict.items() if covis >= 0.7])
    random.shuffle(pairs)
    n = len(pairs)
    pairs = pairs[:max_num_pairs]
    print(f'Loading covisibility data for "{scene}"... kept {len(pairs)} out of {n} covisible pairs')

    # Load the images.
    for pair in tqdm(pairs):
        st = time.time()
        image_0_id, image_1_id = pair.split('-')
        img_path0 = f'{src}/{scene}/images/{image_0_id}.jpg'
        img_path1 = f'{src}/{scene}/images/{image_1_id}.jpg'

        # img_path0, img_path1 = restoration(img_path0, img_path1)

        # print(f'{image_0_id}.jpg')
        # print(f'{image_1_id}.jpg')

        image_0 = load_torch_image(img_path0, device)
        image_1 = load_torch_image(img_path1, device)

        input_dict = {"image0": K.color.rgb_to_grayscale(image_0), 
                "image1": K.color.rgb_to_grayscale(image_1)}

        with torch.no_grad():
            correspondences = matcher(input_dict)
            
        mkpts0 = correspondences['keypoints0'].cpu().numpy()
        mkpts1 = correspondences['keypoints1'].cpu().numpy()
        
        if len(mkpts0) > 7:
            #F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.25, 0.9999, 100000)
            F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.1845, 0.999999, 220000)
            inliers = inliers > 0
            assert F.shape == (3, 3), 'Malformed F?'
            # F_dict[sample_id] = F
        else:
            # F_dict[sample_id] = np.zeros((3, 3))
            continue

        F_str = FlattenMatrix(F)
        sample_ids.append(f'phototourism;{scene};{pair}')
        fund_matrices.append(F_str)

        gc.collect()
        nd = time.time()
        print('Matches: {}'.format(len(mkpts0)))

    if count == folder_limit:
        break

Folder 1/16
Loading covisibility data for "british_museum"... kept 2 out of 102 covisible pairs


  0%|          | 0/2 [00:00<?, ?it/s]

93658023_4980549800.jpg
57504314_2114264842.jpg


 50%|█████     | 1/2 [00:09<00:09,  9.20s/it]

Matches: 3587
77723525_5227836172.jpg
57504314_2114264842.jpg


...

Loading covisibility data for "buckingham_palace"... kept 2 out of 49 covisible pairs


  0%|          | 0/2 [00:00<?, ?it/s]

42572558_3847795752.jpg
23458590_2731063123.jpg


  0%|          | 0/2 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fund_matrices

['8.09659935e-08 3.83725163e-06 -1.68533065e-03 -5.10271305e-06 -2.32134495e-07 6.33474070e-03 2.42883175e-03 -6.35963207e-03 -3.40481219e-01',
 '2.72286005e-06 -1.71651716e-04 4.10358635e-02 1.73362759e-04 -9.96309453e-07 -1.33780179e-01 -4.40917315e-02 1.31653566e-01 1.00000000e+00',
 '2.32940682e-09 -1.04529046e-05 4.36834589e-03 1.05961579e-05 -1.66145650e-07 2.99953328e-04 -5.18184729e-03 -5.67965389e-04 1.24622482e-01',
 '6.91690150e-09 1.64830708e-06 -6.68069937e-04 -6.11293678e-08 1.39626628e-07 4.43462089e-03 3.41584267e-06 -5.14727169e-03 2.65755199e-01',
 '-3.31288429e-07 1.27240775e-05 -5.93650056e-03 -1.27419800e-05 -8.85773132e-07 9.82647537e-03 5.96338569e-03 -8.30647196e-03 -3.62606427e-01',
 '1.81488361e-08 1.81461692e-05 -9.20610804e-03 -1.79502077e-05 -1.73929856e-07 8.73613601e-03 8.07882112e-03 -8.68600412e-03 5.10308514e-01']

In [ ]:
sample_ids

['phototourism;british_museum;77723525_5227836172-42803655_6429621523',
 'phototourism;british_museum;19254317_158538587-18486676_4996206525',
 'phototourism;british_museum;93658023_4980549800-42803655_6429621523',
 'phototourism;british_museum;93658023_4980549800-77723525_5227836172',
 'phototourism;british_museum;77723525_5227836172-69960354_6519615529',
 'phototourism;british_museum;93658023_4980549800-75879177_2453112255']

In [ ]:
df = pd.DataFrame({'sample_id':sample_ids, 'fundamental_matrix':fund_matrices})
df.to_csv('train_pred_nores.csv', index=False)

In [ ]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

print('--- Evaluate prediction ---')
maa, maa_per_scene, errors_dict_q, errors_dict_t = EvaluateSubmission('train_pred_nores.csv', scaling_dict, thresholds_q, thresholds_t)
for scene, cur_maa in maa_per_scene.items():
    print(f'Scene "{scene}" ({len(errors_dict_q[scene])} pairs), mAA={cur_maa:.05f}')
print()
print(f'Full dataset: mAA={maa:.05f}')
print()

--- Evaluate prediction ---


100%|██████████| 6/6 [00:00<00:00, 5856.60it/s]

Scene "british_museum" (6 pairs), mAA=0.13333

Full dataset: mAA=0.13333

